In [1]:
from __future__ import annotations

from egglog import *
from typing import List, Tuple, Optional
egraph = EGraph()

In [2]:
class Num(Expr):
    def __init__(self, value: i64Like) -> None: ...

    @classmethod
    def var(cls, name: StringLike) -> Num: ...

    def __add__(self, other: Num) -> Num: ...

    def __mul__(self, other: Num) -> Num: ...


expr1 = Num(2) * (Num.var("x") + Num(3))
expr2 = Num(6) + Num(2) * Num.var("x")

a, b, c = vars_("a b c", Num)
i, j = vars_("i j", i64)

egraph = EGraph()
egraph.register(expr1, expr2)

egraph.run(
    ruleset(
        rewrite(a + b).to(b + a),
        rewrite(a * (b + c)).to((a * b) + (a * c)),
        rewrite(Num(i) + Num(j)).to(Num(i + j)),
        rewrite(Num(i) * Num(j)).to(Num(i * j)),
    )
    * 10
)

egraph.check(expr1 == expr2)

In [3]:
# now lets show how to use the use the egraph to get the mutations of a simple workflow


class Performer(Expr):
    def __init__(self, name: StringLike) -> None: ...
    
    @classmethod
    def var(cls, name: StringLike) -> Performer: ...
    
class Task(Expr):
    def __init__(self, name: StringLike, performer:Performer) -> None: ...
    
    @classmethod
    def var(cls, name: StringLike) -> Task: ...



class TaskGraph(Expr):
    def __init__(self, dependencies: List[Tuple[Task, Task]]) -> None:
        self.dependencies = dependencies

    @classmethod
    def var(cls, name: StringLike) -> TaskGraph:
        return cls(dependencies=[])

## Process DSL (Egglog) and mutation scaffolding

This section defines a tiny DSL for processes (Tasks, Roles, Persons, dependencies, assignments) and prepares variables and helpers we’ll use to encode mutation rules.

We’ll start with:
- Core types: Task, Role, Person
- Facts: Dep(prev, nxt), AssignRole(task, role), AssignPerson(task, person), Tag(task, label)
- Class and binding helpers: ClassOf(task, klass), EndOfClass(task, klass), BindSamePerson(prev, nxt)
- Task constructors for derived nodes: Task.merge(a,b), Task.split_l(t), Task.split_r(t), Task.qc_of(t), Task.manual_of(t), Task.par1_of(t), Task.par2_of(t), Task.join(t1,t2)

Next, we’ll add rewrite rules for the requested mutations. For now, this cell just introduces the symbols and sets up variables; the following code cell wires a tiny base process to sanity-check the setup.

In [4]:
# NOTE: Earlier scaffold removed to avoid conflicts with the updated DSL below.
# Keeping this cell as a placeholder; see later cells for the active DSL definitions.
_scaffold_removed = True

In [5]:
# Define a tiny base process: T1 -> T2 -> T3, with roles
T1, T2, T3 = Task("T1"), Task("T2"), Task("T3")
H, AIr = Role("Human"), Role("AI")


base = Graph(
    U(
        U(
            U(
                U(
                    Dep(T1, T2),
                    Dep(T2, T3)
                ),
                RoleAt(T1, H)
            ),
            RoleAt(T2, AIr)
        ),
        RoleAt(T3, H)
    )
)


# Register and run a tiny propagation rule for same-person binding through AssignSameAs facts
E.register(base)


E.run(
    ruleset(
        # Same-person binding via DidBy and AssignSameAs
        rewrite(U(U(AssignSameAs(B, A), DidBy(A, X)), C)).to(U(U(AssignSameAs(B, A), DidBy(B, X)), C)),
    ) * 2  # small bound
)


# Simple check that the graph is present
E.check(base == base)

TypeError: Failed to bind arguments for Task with args ('T1',) and kwargs {}: missing a required argument: 'performer'

## Tiny process DSL for mutations (updated design)


- Role: describes what is being done (activity), not who.
- Performer: expressed via facts:
  - AssignPool(task, Pool(name))
  - AssignSameAs(task, Task)


- Core facts (all are `Fact`):
  - Task(name) is an entity, not a Fact
  - Dep(prev, nxt)
  - RoleAt(task, role)
  - AssignPool, AssignSameAs
  - Optional: Tag, ClassOf, EndOfClass, BindSamePerson(prev, nxt), DidBy(task, person)


- Graph wrapper: multiset `U(f1, f2)` over `Fact` with unit `Empty` for AC-normalization.


This avoids union types in annotations (which egglog doesn’t like) and fixes the `U.__init__` resolution by making `U` a Fact with concrete argument sorts.

In [12]:
from egglog import *


# Cleanup: remove stale symbols from earlier cells to avoid type resolution conflicts


for _n in [
    'Fact','Task','Role','Pool','Dep','RoleAt','AssignPool','AssignSameAs','DidBy','Tag','ClassOf','EndOfClass','Empty','U','Graph',
    'AssignPerformer','AssignRole','AssignPerson','BindSamePerson','AllowedChange'
]:


    if _n in globals():


        try:


            del globals()[_n]


        except Exception:


            pass




# Base sort for all graph facts/items
class Fact(Expr):
    def __init__(self) -> None: ...


# Entities
class Task(Expr):
    def __init__(self, name: StringLike) -> None: ...
    @classmethod
    def var(cls, name: StringLike) -> Task: ...


class Role(Expr):
    def __init__(self, name: StringLike) -> None: ...
    @classmethod
    def var(cls, name: StringLike) -> Role: ...


# Performer carriers
class Pool(Expr):
    def __init__(self, name: StringLike) -> None: ...


# Facts (all are Fact)
class Dep(Fact):
    def __init__(self, prev: Task, nxt: Task) -> None: ...


class RoleAt(Fact):
    def __init__(self, t: Task, r: Role) -> None: ...


# Explicit performer assignments to avoid union types
class AssignPool(Fact):
    def __init__(self, t: Task, pool: Pool) -> None: ...


class AssignSameAs(Fact):
    def __init__(self, t: Task, src: Task) -> None: ...


class DidBy(Fact):
    def __init__(self, t: Task, person: StringLike) -> None: ...  # latent person identity


class Tag(Fact):
    def __init__(self, t: Task, label: StringLike) -> None: ...


class ClassOf(Fact):
    def __init__(self, t: Task, klass: StringLike) -> None: ...


class EndOfClass(Fact):
    def __init__(self, t: Task, klass: StringLike) -> None: ...


# Graph multiset (AC + unit)
class Empty(Fact):
    def __init__(self) -> None: ...


class U(Fact):
    def __init__(self, a: Fact, b: Fact) -> None: ...


class Graph(Expr):
    def __init__(self, u: Fact) -> None: ...


E = EGraph()


# Variables
A, B, C = vars_("A B C", Fact)
R = vars_("R", Role)
X = vars_("X", str)  # use Python 'str' for string sort


# AC + unit rules for U (bounded rounds to avoid non-termination)
# Note: we intentionally avoid the commutativity flip U(A,B)->U(B,A) to prevent oscillation.
E.run(
    ruleset(
        rewrite(U(Empty(), A)).to(A),
        rewrite(U(A, Empty())).to(A),
        # Right-associate only; no commutativity flip
        rewrite(U(U(A, B), C)).to(U(A, U(B, C))),
    ) * 3  # bound normalization rounds
)


# Sanity: SameAs propagation — if AssignSameAs(B, A) and DidBy(A, X), imply DidBy(B, X)
E.run(
    ruleset(
        rewrite(U(U(AssignSameAs(B, A), DidBy(A, X)), C)).to(U(U(AssignSameAs(B, A), DidBy(B, X)), C)),
    ) * 2  # small bound
)


# Quick smoke: a tiny process and a latent person identity
T1, T2 = Task("T1"), Task("T2")
base = Graph(U(U(Dep(T1, T2), RoleAt(T1, Role("Review"))), AssignSameAs(T2, T1)))
E.register(base)
E.register(DidBy(T1, "alice"))
E.run(ruleset())  # no-op to apply rules
E.check(base == base)

ERROR! Session/line number was not unique in database. History logging moved to new session 992


## Single-rewrite demo (step-by-step)

Goal: show one rewrite applies and terminates, in isolation from earlier cells.

We define a tiny DSL with just Tasks, two Facts (AssignSameAs, DidBy), and a simple container U(..., ...) plus Graph(...). No AC/commutativity rules are used, so there’s nothing to loop. Then we add exactly one rewrite and run it once.

In [ ]:
# Minimal, isolated example: single rewrite (no normalization rules)
from egglog import *

# 1) Define the minimal language
class Fact(Expr):
    def __init__(self) -> None: ...

class Task(Expr):
    def __init__(self, name: StringLike) -> None: ...

class AssignSameAs(Fact):
    def __init__(self, t: Task, src: Task) -> None: ...

class DidBy(Fact):
    def __init__(self, t: Task, person: StringLike) -> None: ...

class U(Fact):
    def __init__(self, a: Fact, b: Fact) -> None: ...

class Graph(Expr):
    def __init__(self, u: Fact) -> None: ...

# 2) Create a fresh e-graph and variables
E1 = EGraph()
A, B = vars_("A B", Task)
X = vars_("X", str)
C = vars_("C", Fact)

# 3) One rule: if AssignSameAs(B, A) and DidBy(A, X), infer DidBy(B, X)
rule = ruleset(
    rewrite(U(U(AssignSameAs(B, A), DidBy(A, X)), C)).to(
        U(U(AssignSameAs(B, A), DidBy(B, X)), C)
    ),
)

# 4) One concrete graph and an expected, single-step result
T1, T2 = Task("T1"), Task("T2")
lhs = U(
    U(AssignSameAs(T2, T1), DidBy(T1, "alice")),
    U(AssignSameAs(T2, T1), DidBy(T1, "alice")),
)
base = Graph(lhs)

rhs = U(
    U(AssignSameAs(T2, T1), DidBy(T2, "alice")),
    U(AssignSameAs(T2, T1), DidBy(T1, "alice")),
)
expected = Graph(rhs)

# Register both so the equality can be proven in the same e-graph
E1.register(base, expected)

# 5) Run the rule exactly once (bounded)
E1.run(rule * 1)

# 6) Check that the rewrite makes base equivalent to expected
E1.check(base == expected)  # proves a single rewrite applied and terminated